In [3]:
import pandas as pd
import numpy as np 
import math
from math import sqrt
import matplotlib.pyplot as plt
import collections
import warnings

import seaborn as sns
warnings.filterwarnings("ignore")

#Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

# Show all columns of dataframe
pd.set_option("display.max_columns", None)

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
df = pd.read_csv("")

# 1. Data Engineering

In [5]:
# Create new ID column and Main Position
df["ID"] = df["Player"] + " (" + df["Team"] + ")"# + " " + str(df["Age"]) + ")"
df["Main Position"] = df["Position"].str.split(",", expand=True)[0]

# Place ID as first column
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df[cols]

df.head(1)

,ID,Main Position,Player,Team,League,Position,Age,Market value,Contract expires,Matches played,Minutes played,Goals,xG,Assists,xA,Duels per 90,"Duels won, %",Birth country,Passport country,Foot,Height,Weight,On loan,Successful defensive actions per 90,Defensive duels per 90,"Defensive duels won, %",Aerial duels per 90,"Aerial duels won, %",Sliding tackles per 90,PAdj Sliding tackles,Shots blocked per 90,Interceptions per 90,PAdj Interceptions,Fouls per 90,Yellow cards,Yellow cards per 90,Red cards,Red cards per 90,Successful attacking actions per 90,Goals per 90,Non-penalty goals,Non-penalty goals per 90,xG per 90,Head goals,Head goals per 90,Shots,Shots per 90,"Shots on target, %","Goal conversion, %",Assists per 90,Crosses per 90,"Accurate crosses, %",Crosses from left flank per 90,"Accurate crosses from left flank, %",Crosses from right flank per 90,"Accurate crosses from right flank, %",Crosses to goalie box per 90,Dribbles per 90,"Successful dribbles, %",Offensive duels per 90,"Offensive duels won, %",Touches in box per 90,Progressive runs per 90,Accelerations per 90,Received passes per 90,Received long passes per 90,Fouls suffered per 90,Passes per 90,"Accurate passes, %",Forward passes per 90,"Accurate forward passes, %",Back passes per 90,"Accurate back passes, %",Lateral passes per 90,"Accurate lateral passes, %",Short / medium passes per 90,"Accurate short / medium passes, %",Long passes per 90,"Accurate long passes, %","Average pass length, m","Average long pass length, m",xA per 90,Shot assists per 90,Second assists per 90,Third assists per 90,Smart passes per 90,"Accurate smart passes, %",Key passes per 90,Passes to final third per 90,"Accurate passes to final third, %",Passes to penalty area per 90,"Accurate passes to penalty area, %",Through passes per 90,"Accurate through passes, %",Deep completions per 90,Deep completed crosses per 90,Progressive passes per 90,"Accurate progressive passes, %",Conceded goals,Conceded goals per 90,Shots against,Shots against per 90,Clean sheets,"Save rate, %",xG against,xG against per 90,Prevented goals,Prevented goals per 90,Back passes received as GK per 90,Exits per 90,Aerial duels per 90.1,Free kicks per 90,Direct free kicks per 90,"Direct free kicks on target, %",Corners per 90,Penalties taken,"Penalty conversion, %"
0,F. Bustos (Internacional),RB,F. Bustos,Internacional,Argentina 1,RB,26,6000000,31.12.24,23,2228,1,0.27,2,1.44,14.87,54.62,Argentina,Argentina,right,167,66,no,8.56,5.49,62.5,1.49,48.65,0.89,1.24,0.16,4.24,5.91,0.36,1,0.04,0,0.0,3.23,0.04,1,0.04,0.01,0,0.0,6,0.24,66.67,16.667,0.08,2.67,37.88,0.0,0.0,2.63,38.46,0.4,2.5,72.58,5.17,57.81,1.41,1.9,0.73,42.09,3.19,1.25,56.43,80.24,21.49,73.31,6.99,87.28,18.38,87.25,48.03,86.04,5.86,51.03,18.16,33.28,0.06,0.57,0.0,0.0,0.44,54.55,0.44,7.39,75.41,2.63,49.23,0.48,33.33,0.57,0.97,12.44,74.68,0,NaN,0,NaN,0,0.0,0.0,0.0,NaN,NaN,0.28,NaN,NaN,0.0,0.0,0.0,0.08,0,0.0


In [7]:
# Drop duplicates and reset index
df.drop_duplicates(subset = ["ID"],inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(2)
df.shape

(5585, 117)

In [8]:
# Drop unnecessary columns
df.drop(columns=["Conceded goals", "Conceded goals per 90",
                "Shots against", "Shots against per 90", "Clean sheets", "Save rate, %", "xG against",
                "xG against per 90", "Prevented goals", "Prevented goals per 90", "Exits per 90",
                "Back passes received as GK per 90", "Aerial duels per 90.1", "Free kicks per 90",
                "Direct free kicks per 90", "Direct free kicks on target, %", "Corners per 90", 
                "Penalties taken", "Penalty conversion, %"], axis=1, inplace=True)

### Create additional metrics 

In [10]:
# Ball protection and offensive duels 
df["Ball Protection"] = df["Offensive duels per 90"] - df["Dribbles per 90"]
df["Ball Protection Succ"] = ((df["Offensive duels per 90"] * df["Offensive duels won, %"])/100 - (df["Dribbles per 90"] * df["Successful dribbles, %"])/100).round(2)
df["Ball Protection, %"] = ((df["Ball Protection Succ"] / df["Ball Protection"])*100).round(2)
#print(df.sort_values(by="Ball Protection, %", ascending=False).head(50))

# Turnover (lost offensive duels incl. dribbles)
df["Turnover"] = -(df["Offensive duels per 90"] * ((100-df["Offensive duels won, %"])/100)).round(2)

# Passing tendency 
df["Passing Tendency"] = (df["Forward passes per 90"] / (df["Back passes per 90"] + df["Lateral passes per 90"])).round(2)
#df.sort_values(by="Passing Tendency", ascending=False).head(50)

# Proactivity (Fouls?)
df["Proactivity"] = df["PAdj Interceptions"] / (df["Shots blocked per 90"] + df["PAdj Sliding tackles"])

# Successful dribbles 
df["Successful dribbles per 90"] = df["Dribbles per 90"] * df["Successful dribbles, %"]

df["Fouls per 90"] = -(df["Fouls per 90"])

#df["Fouls per Duel"] = df["Fouls per 90"] / df["Defensive duels per 90"]


In [11]:
df["Proactivity"][df["Proactivity"] == inf] = 0
df["Proactivity"][numpy.isinf(df["Proactivity"])] = 0

In [12]:
df.fillna(0, inplace=True)
#df.isnull().sum()

### Create Percentiles (only used for Tableau charts)

In [14]:
# Create a list of all leagues to subset dataframes
leagues = df["League"].unique().tolist()

In [15]:
# Initiate an empty dictionary that includes all new dataframes
d={}

df_per = df[(df["Main Position"] == "RB") | (df["Main Position"] == "LB")]

for league in leagues:
    d[league] = df_per[df_per["League"] == league]

In [16]:
# Run for loop to calculate percentiles per metric per dataframe 
for key in d.keys():
    for i in d[key].iloc[:,23:]:
        d[key][i] = ((d[key][i].rank(pct=True))*100).round(2)

# 2. Preprocessing the Data 

In [17]:
kpi1 = "PAdj Interceptions"
kpi2 = "Aerial duels won, %"
kpi3 = "Defensive duels won, %"
kpi4 = "Fouls per 90"
kpi5 = "Deep completions per 90"
kpi6 = "Successful dribbles, %"
kpi7 = "Progressive runs per 90"
kpi8 = "Accurate short / medium passes, %"
kpi9 = "Turnover"
kpi10 = "Progressive passes per 90"

w1 = 0.1
w2 = 0.05
w3 = 0.1
w4 = 0.1
w5 = 0.05
w6 = 0.05
w7 = 0.1
w8 = 0.2
w9 = 0.15
w10 = 0.1

In [18]:
# Define Stats
cols_stats = [kpi1, kpi2, kpi3, kpi4, kpi5, kpi6, kpi7, kpi8, kpi9, kpi10]               

In [19]:
# Create a dataframe of numerical values by selecting all numerical columns
df_stats = df[cols_stats]
df_stats.head()

,PAdj Interceptions,"Aerial duels won, %","Defensive duels won, %",Fouls per 90,Deep completions per 90,"Successful dribbles, %",Progressive runs per 90,"Accurate short / medium passes, %",Turnover,Progressive passes per 90
0,5.91,48.65,62.50,-0.36,0.57,72.58,1.90,86.04,-2.18,12.44
1,6.45,53.15,68.21,-0.92,0.32,50.00,0.60,86.19,-1.43,7.51
2,9.03,57.32,65.11,-1.72,0.36,56.00,1.70,87.02,-4.65,9.23
3,7.37,42.66,63.16,-1.11,0.34,59.38,1.50,87.43,-4.43,6.94
4,6.51,41.67,65.16,-1.23,0.52,56.76,0.95,82.37,-2.13,6.76


In [20]:
# Check for NULL values
df_stats.fillna(0, inplace=True)
#df_stats.isnull().sum()

In [21]:
# Standardization
x = df_stats
scaler = StandardScaler()

# Option2
x[cols_stats] = scaler.fit_transform(x)

In [22]:
europe = ["Austria 1", "Austria 2", "Belgium 1", "Belgium 2", "Croatia", "Czech 1", "Czech 2", "Denmark 1",
         "Denmark 2", "England 1", "England 2", "England 3", "Finland", "France 1", "France 2", "France 3",
         "Germany 1", "Germany 2", "Germany 3", "Greece 1", "Hungary 1", "Italy 1", "Italy 2", "Netherlands 1",
         "Netherlands 2", "Norway", "Poland 1", "Poland 2", "Portugal 1", "Portugal 2", "Romania 1",
         "Romania 2", "Russia 1", "Russia 2", "Scotland 1", "Scotland 2", "Serbia", "Slovakia", "Slovenia",
         "Spain 1", "Spain 2", "Sweden 1", "Sweden 2", "Switzerland 1", "Switzerland 2", 
         "Turkey 1", "Turkey 2", "Ukraine 1", "Ukraine 2"]

# 3. Calculate Similarity 

In [23]:
def calculate_similarity(player, squad, value, age):
    
    dist1 = float(df[kpi1][(df["ID"] == player + " (" + squad + ")")])
    dist2 = float(df[kpi2][(df["ID"] == player + " (" + squad + ")")])
    dist3 = float(df[kpi3][(df["ID"] == player + " (" + squad + ")")])
    dist4 = float(df[kpi4][(df["ID"] == player + " (" + squad + ")")])
    dist5 = float(df[kpi5][(df["ID"] == player + " (" + squad + ")")])
    dist6 = float(df[kpi6][(df["ID"] == player + " (" + squad + ")")])
    dist7 = float(df[kpi7][(df["ID"] == player + " (" + squad + ")")])
    dist8 = float(df[kpi8][(df["ID"] == player + " (" + squad + ")")])
    dist9 = float(df[kpi9][(df["ID"] == player + " (" + squad + ")")])
    dist10 = float(df[kpi10][(df["ID"] == player + " (" + squad + ")")])
                    
    df["Dist"] = np.sqrt( w1*((dist1 - df[kpi1])**2) + w2*((dist2 - df[kpi2])**2)
                        + w3*((dist3 - df[kpi3])**2) + w4*((dist4 - df[kpi4])**2)  
                        + w5*((dist5 - df[kpi5])**2) + w6*((dist6 - df[kpi6])**2) 
                        + w7*((dist7 - df[kpi7])**2) + w8*((dist8 - df[kpi8])**2) 
                        + w9*((dist9 - df[kpi9])**2) + w10*((dist10 - df[kpi10])**2)
                        )
    
    df["Similarity"] = 1 - df["Dist"]/np.max(df["Dist"])
    
    dff = df[(df["Market value"] < value) & (df["Age"] < age) & (df["Main Position"] == "RB")]
    dff = dff.loc[dff['League'].isin(europe)]
    
    return dff[["ID", "Age", "League", "Market value", "Position", "Dist", "Similarity", kpi1, kpi2, kpi3, kpi4, kpi5, kpi6, kpi7, kpi8, kpi9]].sort_values(by="Similarity", ascending=False).reset_index(drop=True).head(50)

In [24]:
calculate_similarity("K. Walker", "Manchester City", 5000000, 25)

,ID,Age,League,Market value,Position,Dist,Similarity,PAdj Interceptions,"Aerial duels won, %","Defensive duels won, %",Fouls per 90,Deep completions per 90,"Successful dribbles, %",Progressive runs per 90,"Accurate short / medium passes, %",Turnover
0,L. Rota (AEK Athens),24,Greece 1,1000000,RB,2.198713,0.955853,4.78,53.85,61.03,-1.04,0.31,60.71,2.03,93.12,-2.60
1,A. Sampsted (Bodø / Glimt),24,Norway,1200000,RB,2.371027,0.952393,7.37,54.84,64.91,-0.58,1.23,55.88,6.08,91.63,-2.82
2,D. Haruţ (FCS Bucureşti),23,Romania 1,1000000,"RB, CB, LCB",2.375406,0.952305,5.11,50.00,63.24,-0.78,0.37,59.09,0.78,90.90,-1.64
3,S. Boey (Galatasaray),21,Turkey 1,4000000,RB,2.499791,0.949808,5.73,48.48,60.50,-1.46,0.31,52.94,2.15,90.93,-2.53
4,B. Anang (Žilina),22,Slovakia,250000,"RB, LB",2.504224,0.949719,8.09,47.62,67.54,-0.56,0.84,51.52,1.50,92.77,-1.89
5,A. Jallow (IFK Göteborg),24,Sweden 1,1000000,"RB, LB",2.948047,0.940807,5.65,55.22,61.24,-0.98,0.40,53.68,2.16,88.59,-2.85
6,E. Kurtulus (Hammarby),22,Sweden 1,1500000,"RB, RCB",3.003729,0.939689,4.73,57.38,64.16,-0.85,0.99,63.64,1.50,92.55,-4.12
7,N. Markelo (Excelsior),23,Netherlands 2,250000,RB,3.063230,0.938495,5.25,54.84,68.24,-1.27,0.38,52.38,1.95,90.32,-4.07
8,R. Zeegers (MVV),22,Netherlands 2,275000,"RB, RCB",3.247810,0.934788,5.09,45.45,60.17,-0.94,0.28,50.00,1.41,88.94,-1.22
9,S. Floranus (Antalyaspor),23,Turkey 1,1000000,RB,3.262445,0.934495,5.37,50.00,58.67,-0.88,0.62,55.56,0.44,89.31,-1.15
